![ML Logo](https://kaggle2.blob.core.windows.net/competitions/kaggle/3948/logos/front_page.png)
# **Bike Rentals in Washington DC**

El objetivo de este ejercicio es poder predecir la cantidad de alquielers de bicicletas en Washington DC en funcion de varios campos.
Contamos con un set de entrenamiento con la siguiente informacion:

#### **datetime** - hourly date + timestamp  
#### **season** -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
#### **holiday** - whether the day is considered a holiday
#### **workingday** - whether the day is neither a weekend nor holiday
#### **weather** - 1: Clear, Few clouds, Partly cloudy, Partly cloudy 
#### 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist 
#### 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds 
#### 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
#### **temp** - temperature in Celsius
#### **atemp** - "feels like" temperature in Celsius
#### **humidity** - relative humidity
#### **windspeed** - wind speed
#### **casual** - number of non-registered user rentals initiated
#### **registered** - number of registered user rentals initiated
#### **count** - number of total rentals

In [3]:
# Load the Data, check number of records, check the format of our records.
rawData = sc.textFile('data/bikes/bike_rentals_no_header.csv', 4)
print "Number of records:" + str(rawData.count())
print rawData.first()

Number of records:10886
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0,3,13,16


In [4]:
# This is our Feature Engineering Section
# Feel free to experiment here
raw = '2011-01-01 10:11:12,4,1,1,3,9.84,14.395,81,0,3,13,16'   
def convert_to_tuple(raw):
    v = raw.split(',')
    print v
    date = v[0].split(' ')
    time = date[1].split(':')
    print date 
    print time
    hour = time[0]
    season = v[1]
    season_vec = [0,0,0,0]
    season_vec[int(season)-1]=1
    isHoliday = int(v[2])
    isWorkingDay = int(v[3])
    weather = v[4]
    weather_vec = [0,0,0,0]
    weather_vec[int(weather)-1]=1
    temp = float(v[5])
    atemp = float(v[6])
    humidity = float(v[7])
    windspeed = float(v[8])
    count = int(v[11])
    hour_vec = [0]*23
    hour = int(hour)
    if hour==0:
        hour=23
    hour = hour - 1
    hour_vec[hour]=1
    #record = [count,isHoliday,isWorkingDay,temp,atemp,humidity,windspeed]+season_vec+weather_vec+hour_vec
    record = [count,isHoliday,isWorkingDay,temp]+season_vec+weather_vec+hour_vec
    #record = [count,temp]+hour_vec
    record = tuple(record)
    return record

In [5]:
# Format the records, split by comma, then format the date and time field into several fields: year, month, day
# hour and minutes.

   
# First step: split by comma
rdd = rawData.map(lambda x:convert_to_tuple(x))
print rdd.take(1)

[(16, 0, 0, 9.84, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1)]


In [6]:
# Convert each row to a Labeled Point
from pyspark.mllib.regression import LabeledPoint

dataSet = rdd.map(lambda x: LabeledPoint(x[0],x[1:]))
print dataSet.take(1)
            

[LabeledPoint(16.0, [0.0,0.0,9.84,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0])]


In [25]:
# We can use dataSet or normalData
# Split the dataset in train, validation and test sets (80,10,10)
weights = [.8, .1, .1]
seed = 42 #of course
train, validation, test = dataSet.randomSplit(weights=weights,seed=seed)
nTrain = train.count()
nVal = validation.count()
nTest = test.count()

print nTrain, nVal, nTest, nTrain + nVal + nTest
print dataSet.count()

8760 1058 1068 10886
10886


In [8]:
# Now normalize the dataset
from pyspark.mllib.feature import StandardScaler, StandardScalerModel
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.util import MLUtils

labelTrain = train.map(lambda x: x.label)
featuresTrain = train.map(lambda x: x.features)

labelValidation = validation.map(lambda x: x.label)
featuresValidation = validation.map(lambda x: x.features)

labelTest = test.map(lambda x: x.label)
featuresTest = test.map(lambda x: x.features)

# The normalizer is created only on the train set, then applied to train, validation and test sets!
scaler2 = StandardScaler(withMean=True, withStd=True).fit(featuresTrain)

train = labelTrain.zip(scaler2.transform(featuresTrain.map(lambda x: Vectors.dense(x.toArray()))))
train = train.map(lambda x: LabeledPoint(x[0],x[1:]))

validation = labelValidation.zip(scaler2.transform(featuresValidation.map(lambda x: Vectors.dense(x.toArray()))))
validation = validation.map(lambda x: LabeledPoint(x[0],x[1:]))

test = labelTest.zip(scaler2.transform(featuresTest.map(lambda x: Vectors.dense(x.toArray()))))
test = test.map(lambda x: LabeledPoint(x[0],x[1:]))


train.cache()
validation.cache()
test.cache()

print train.take(1)

[LabeledPoint(16.0, [-0.171034841052,-1.45964532572,-1.33441520198,1.74523397799,-0.577668801449,-0.576087108746,-0.582589728142,0.715801957628,-0.593840536403,-0.290777313979,-0.0106843460793,-0.209395035961,-0.209691840999,-0.201856903343,-0.20670783676,-0.209988297125,-0.209988297125,-0.207905723831,-0.20670783676,-0.209395035961,-0.208800373093,-0.207307502895,-0.210580168632,-0.209988297125,-0.207307502895,-0.210875586984,-0.209395035961,-0.207606793219,-0.20850251221,-0.209988297125,-0.207905723831,-0.20850251221,-0.209691840999,3.32638729664])]


In [9]:
# Baseline model
# We are going to take the average number of bikes as our baseline model
# and evaluate the RMSE error based on this baseline
trainAvg = train.map(lambda x:x.label).mean()
validationAvg = validation.map(lambda x:x.label).mean()
testAvg = test.map(lambda x:x.label).mean()
#print averageTrainYear.take(2)
print trainAvg
print validationAvg
print testAvg

191.173515982
201.011342155
185.511235955


In [10]:
# Function to evaluate RMSE
import math

def squaredError(label, prediction):
    return (label-prediction)**2

def calcRMSE(labelsAndPreds):
    l2 = (labelsAndPreds.map(lambda x:squaredError(x[0],x[1])).reduce(lambda x,y:x+y)) / labelsAndPreds.count()
    return math.sqrt(l2)

labelsAndPreds = sc.parallelize([(3., 1.), (1., 2.), (2., 2.)])
#RMSE = sqrt[((3-1)^2 + (1-2)^2 + (2-2)^2) / 3] = 1.291
exampleRMSE = calcRMSE(labelsAndPreds)
print exampleRMSE

1.29099444874


In [26]:
# Evaluate RMSE for baseline
labelsAndPredsTrain = train.map(lambda x:(x.label,trainAvg))
#print labelsAndPredsTrain.take(1)
rmseTrainBase = calcRMSE(labelsAndPredsTrain)

labelsAndPredsVal = validation.map(lambda x:(x.label,trainAvg))
rmseValBase = calcRMSE(labelsAndPredsVal)

labelsAndPredsTest = test.map(lambda x:(x.label,trainAvg))
rmseTestBase = calcRMSE(labelsAndPredsTest)

print 'Our models have to be better than this!'
print 'Baseline Train RMSE = {0:.6f}'.format(rmseTrainBase)
print 'Baseline Validation RMSE = {0:.6f}'.format(rmseValBase)
print 'Baseline Test RMSE = {0:.6f}'.format(rmseTestBase)

Our models have to be better than this!
Baseline Train RMSE = 181.406548
Baseline Validation RMSE = 181.899555
Baseline Test RMSE = 178.138314


In [12]:
# Use Mlib Linear Regression

from pyspark.mllib.regression import LinearRegressionWithSGD

# Values to use when training the linear regression model
numIters = 500  # iterations
alpha = 1 # step
miniBatchFrac = 1.0  # miniBatchFraction
reg = 1e-1  # regParam
regType = 'l2'  # regType
useIntercept = True  # intercept


In [13]:
# Train!

firstModel = LinearRegressionWithSGD.train(train,regType= regType, iterations=numIters, step=alpha,
regParam=reg, miniBatchFraction=miniBatchFrac,intercept=useIntercept,
)

# weightsLR1 stores the model weights; interceptLR1 stores the model intercept
#print firstModel
weightsLR1 = firstModel.weights
interceptLR1 = firstModel.intercept
print weightsLR1, interceptLR1

[-1.35907580734,0.786781122118,43.0523595536,-13.9360880159,4.38706745337,-1.88123437396,11.322247504,9.27559730283,0.561947017677,-17.2421298357,-1.30370121931,-27.0688558924,-29.2999702902,-30.243696226,-31.2995714576,-28.938152565,-18.2230117472,6.08003464408,32.9551889338,6.77540150715,-3.84262084844,2.25576541657,9.16420108612,10.200625209,6.14891981041,9.05082699411,20.5052679088,48.5569701229,42.4912273942,21.1429463314,6.20709530565,-3.45653993689,-10.364828855,-28.4608831084] 173.802353182


In [14]:
labelsAndPreds = train.map(lambda x:(x.label,firstModel.predict(x.features)))

labelsAndPreds = validation.map(lambda x:(x.label,firstModel.predict(x.features)))
labelsAndPreds = labelsAndPreds.map(lambda x:(x[0],0) if x[1] < 0 else (x[0],x[1]))

print labelsAndPreds.take(10)
rmseValLR0 = calcRMSE(labelsAndPreds)

print ('Validation RMSE:\n\tBaseline = {0:.3f}\n\tLR0 = {1:.3f}').format(rmseValBase, rmseValLR0)

[(8.0, 260.98400830297697), (36.0, 107.35381608375793), (35.0, 264.37076978963159), (36.0, 143.92564946924654), (9.0, 0), (3.0, 0), (88.0, 112.05153470533529), (157.0, 340.39183927576562), (4.0, 0), (48.0, 74.609628187551536)]
Validation RMSE:
	Baseline = 181.633
	LR0 = 113.532


In [15]:
# Grid Search for regularization param
bestRMSE = rmseValLR0
bestRegParam = reg
bestModel = firstModel

numIters = 500
alpha = 1.0
miniBatchFrac = 1.0
for reg in [0.01,0.001,0.0001]:
    model = LinearRegressionWithSGD.train(train, numIters, alpha,
                                          miniBatchFrac, regParam=reg,
                                          regType='l2', intercept=True)
    labelsAndPreds = validation.map(lambda lp: (lp.label, model.predict(lp.features)))
    labelsAndPreds = labelsAndPreds.map(lambda x:(x[0],0) if x[1] < 0 else (x[0],x[1]))
    rmseValGrid = calcRMSE(labelsAndPreds)
    print reg,rmseValGrid

    if rmseValGrid < bestRMSE:
        bestRMSE = rmseValGrid
        bestRegParam = reg
        bestModel = model
rmseValLRGrid = bestRMSE

print ('Validation RMSE:\n\tBaseline = {0:.3f}\n\tLR0 = {1:.3f}'+
       '\n\tLRGrid = {3:.3f}').format(rmseValBase,rmseValLR0,3,rmseValLRGrid)

0.01 111.324271881
0.001 111.275892742
0.0001 111.273287768
Validation RMSE:
	Baseline = 181.633
	LR0 = 113.532
	LRGrid = 111.273


In [18]:
# Grid Search for alpha and numInters
reg = bestRegParam
modelRMSEs = []
bestalpha = 1
bestnumiters = 100

for alpha in [2,2.5,5]:
    for numIters in [2000,5000,7000]:
        model = LinearRegressionWithSGD.train(train, numIters, alpha,
                                              miniBatchFrac, regParam=reg,
                                              regType='l2', intercept=True)
        labelsAndPreds = validation.map(lambda lp: (lp.label, model.predict(lp.features)))
        labelsAndPreds = labelsAndPreds.map(lambda x:(x[0],0) if x[1] < 0 else (x[0],x[1]))
        rmseVal = calcRMSE(labelsAndPreds)
        print 'alpha = {0:.0e}, numIters = {1}, RMSE = {2:.3f}'.format(alpha, numIters, rmseVal)
        modelRMSEs.append(rmseVal)
        if rmseVal < bestRMSE:
            bestRMSE = rmseVal
            bestRegParam = reg
            bestModel = model
            bestalpha = alpha
            bestnumiters = numIters

print bestRMSE            

alpha = 2e+00, numIters = 2000, RMSE = 111.266
alpha = 2e+00, numIters = 5000, RMSE = 111.266
alpha = 2e+00, numIters = 7000, RMSE = 111.266
alpha = 2e+00, numIters = 2000, RMSE = 111.268
alpha = 2e+00, numIters = 5000, RMSE = 111.268
alpha = 2e+00, numIters = 7000, RMSE = 111.268
alpha = 5e+00, numIters = 2000, RMSE = 111.271
alpha = 5e+00, numIters = 5000, RMSE = 111.271
alpha = 5e+00, numIters = 7000, RMSE = 111.271
111.266322577


In [19]:
# Now add two-way interactions!
import numpy as np
import itertools

def twoWayInteractions(lp):
    """Creates a new `LabeledPoint` that includes two-way interactions.

    Note:
        For features [x, y] the two-way interactions would be [x^2, x*y, y*x, y^2] and these
        would be appended to the original [x, y] feature list.

    Args:
        lp (LabeledPoint): The label and features for this observation.

    Returns:
        LabeledPoint: The new `LabeledPoint` should have the same label as `lp`.  Its features
            should include the features from `lp` followed by the two-way interaction features.
    """
    print lp.features
    ll = [x*y for x in lp.features for y in lp.features]
    return LabeledPoint(lp.label,np.hstack((lp.features,ll)))
    

print twoWayInteractions(LabeledPoint(0.0, [2,4, 3]))

# Transform the existing train, validation, and test sets to include two-way interactions.
#print parsedTrainData.take(1)
dataInt = dataSet.map(lambda x:twoWayInteractions(x))
dataInt.cache()
print dataInt.take(1)

[2.0,4.0,3.0]
(0.0,[2.0,4.0,3.0,4.0,8.0,6.0,8.0,16.0,12.0,6.0,12.0,9.0])
[LabeledPoint(16.0, [0.0,0.0,9.84,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96.8256,9.84,0.0,0.0,0.0,9.84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.84,0.0,0.0,9.84,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [20]:
# Normalizar!
label = dataInt.map(lambda x: x.label)
features = dataInt.map(lambda x: x.features)

scaler2 = StandardScaler(withMean=True, withStd=True).fit(features)
normalDataInt = label.zip(scaler2.transform(features.map(lambda x: Vectors.dense(x.toArray()))))
normalDataInt = normalDataInt.map(lambda x: LabeledPoint(x[0],x[1:]))
print normalDataInt.take(1)



[LabeledPoint(16.0, [-0.171482599437,-1.46060522521,-1.3335994358,1.74716521159,-0.578950117465,-0.578950117465,-0.579091541309,0.716644311437,-0.593236810864,-0.292678705928,-0.00958441996199,-0.208604747934,-0.207162151341,-0.203518327358,-0.205711113405,-0.208124809297,-0.208844372511,-0.208844372511,-0.208844372511,-0.208844372511,-0.208844372511,-0.208844372511,-0.209083768332,-0.209083768332,-0.209083768332,-0.209083768332,-0.209083768332,-0.209083768332,-0.209083768332,-0.209083768332,-0.209083768332,-0.209083768332,-0.209083768332,3.30885179535,-0.171482599437,0.0,-0.157056646907,-0.0810206949822,-0.0665466345763,-0.0943202505801,-0.0943202505801,-0.138187528214,-0.0923172286409,-0.0371441935226,0.0,-0.0345761818629,-0.0345761818629,-0.0332181934849,-0.0345761818629,-0.0345761818629,-0.0345761818629,-0.0345761818629,-0.0345761818629,-0.0345761818629,-0.0345761818629,-0.0345761818629,-0.0345761818629,-0.0345761818629,-0.0345761818629,-0.0345761818629,-0.0345761818629,-0.03457618

In [21]:
weights = [.8, .1, .1]
seed = 42 #of course
trainInt, validationInt, testInt = normalDataInt.randomSplit(weights,seed=seed)
trainInt.cache()
validationInt.cache()
testInt.cache()
print trainInt.count()

8760


In [22]:
# build model for 2-way interactions
numIters = 500
alpha = 0.01
miniBatchFrac = 1.0
reg = 1e-8
bestInteractModel = ''


for reg in [1e-12,1e-10,1e-8]:
    for alpha in [0.05,0.15,0.1,0.2]:
        modelInteract = LinearRegressionWithSGD.train(trainInt, numIters, alpha,
                                                      miniBatchFrac, regParam=reg,
                                                      regType='l2', intercept=True)
        labelsAndPredsInteract = validationInt.map(lambda lp: (lp.label, modelInteract.predict(lp.features)))
        labelsAndPreds = labelsAndPreds.map(lambda x:(x[0],0) if x[1] < 0 else (x[0],x[1]))
        rmseValInteract = calcRMSE(labelsAndPredsInteract)
        
        if rmseValInteract < bestRMSE:
            bestRMSE = rmseValInteract
            bestRegParam = reg
            bestAlpha = alpha
            bestInteractModel = modelInteract
            print '*******'
            print alpha,reg,bestRMSE
        else:
            print alpha,reg,bestRMSE

print bestRMSE

*******
0.05 1e-12 101.545113854
*******
0.15 1e-12 86.491764933
0.1 1e-12 86.491764933
*******
0.2 1e-12 85.4662617465
0.05 1e-10 85.4662617465
0.15 1e-10 85.4662617465
0.1 1e-10 85.4662617465
0.2 1e-10 85.4662617465
0.05 1e-08 85.4662617465
0.15 1e-08 85.4662617465
0.1 1e-08 85.4662617465
0.2 1e-08 85.4662617465
85.4662617465


In [23]:
# Now train that model for longer time
numIters = 5000
alpha = bestAlpha
reg = bestRegParam
modelInteract = LinearRegressionWithSGD.train(trainInt, numIters, alpha,
                                                      miniBatchFrac, regParam=reg,
                                                      regType='l2', intercept=True)
labelsAndPredsInteract = validationInt.map(lambda lp: (lp.label, modelInteract.predict(lp.features)))
labelsAndPreds = labelsAndPreds.map(lambda x:(x[0],0) if x[1] < 0 else (x[0],x[1]))
rmseValInteract = calcRMSE(labelsAndPredsInteract)
print rmseValInteract

85.4662617465


In [24]:
# Evaluate model on test set
print "-Model 1-------------------------------------------"
labelsAndPreds = test.map(lambda lp: (lp.label, bestModel.predict(lp.features)))
labelsAndPreds = labelsAndPreds.map(lambda x:(x[0],0) if x[1] < 0 else (x[0],x[1]))
#print labelsAndPreds.take(15)
rmseTest = calcRMSE(labelsAndPreds)
print rmseTest
print "-Model 2-------------------------------------------"
labelsAndPreds = testInt.map(lambda lp: (lp.label, modelInteract.predict(lp.features)))
labelsAndPreds = labelsAndPreds.map(lambda x:(x[0],0) if x[1] < 0 else (x[0],x[1]))
#print labelsAndPreds.take(15)
results = labelsAndPreds.collect()
rmseTest = calcRMSE(labelsAndPreds)
print rmseTest
for r in results:
    print str(r[0]) + '\t' + str(r[1])

-Model 1-------------------------------------------
107.710156986
-Model 2-------------------------------------------
78.0044569988
106.0	282.061459268
39.0	79.7887229145
22.0	89.0232304724
3.0	0
76.0	131.918042304
52.0	71.2494638103
179.0	341.999997696
182.0	280.234776238
54.0	124.862483571
195.0	357.625680733
71.0	90.5262972321
2.0	0
122.0	157.796843296
1.0	0
87.0	101.945671647
15.0	1.83614534803
62.0	170.492044543
34.0	21.097768752
4.0	0
1.0	0
1.0	0
72.0	216.380204883
41.0	52.1988415226
38.0	37.9552469754
1.0	0
31.0	23.4916877077
12.0	0
52.0	50.8740385615
63.0	44.078402731
95.0	92.5903243358
1.0	0
39.0	40.4281958352
33.0	43.1709938103
83.0	70.7628755176
117.0	148.122313021
53.0	44.3680457343
159.0	300.183528535
89.0	226.725025359
128.0	207.879869706
84.0	92.9571235986
41.0	62.7273301704
26.0	46.6354305265
93.0	206.346699668
5.0	0
33.0	125.041724111
27.0	49.2784547197
104.0	175.295143877
8.0	0
2.0	0
52.0	48.0169240183
63.0	81.9838093026
51.0	38.3512564863
99.0	135.92821706
4.0	0
52.0